<a href="https://colab.research.google.com/github/PurvaChiniya/Aspect-based-sentiment-analysis/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install dependencies
!pip uninstall -y tensorflow
!pip install transformers
import json
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import transformers
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from transformers import BertModel, BertTokenizer

import logging
logging.basicConfig(level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore")
class Model(nn.Module):
  # same as train 
  def __init__(self, BERT_MODEL):
    super(Model, self).__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, 3) # Number of output classes = 3

  def forward(self, ids, mask, token_type_ids):
    last_hidden_state, pooled_output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
    output = self.drop(pooled_output)
    return self.out(output)

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 3.4 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 45.2 MB/s 
     |████████████████████████████████| 61 kB 462 kB/s 
     |████████████████████████████████| 596 kB 48.9 MB/s 
     |████████████████████████████████| 3.3 MB 40.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [13]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
# declare parameters
max_length = 160
batch_size = 32
locations = ['LOCATION1', 'LOCATION2']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety','shopping', 'touristy', 'transit-location']
label_to_int = {
    'Positive': 0,
    'Negative': 1,
    'None': 2
}
# load data
test_data = pd.read_csv('/content/testing_set.csv')
#convert text sentiment to int labels
test_data["sentiment"]=test_data.apply(lambda row:label_to_int[row.sentiment],axis=1)
# drop values
test_data = test_data.reset_index(drop=True)[:400]

tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased',do_lowercase=True)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
class SentiHood:
# same as model with opinions id
  def __init__(self, opinions_id, text, auxiliary_sentence, targets, tokenizer, max_len):
    self.opinions_id = opinions_id
    self.text = text
    self.auxiliary_sentence = auxiliary_sentence
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.targets = targets

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, item):
    opinions_id = self.opinions_id[item]
    text = str(self.text[item])
    auxiliary_sentence = str(self.auxiliary_sentence[item])
    targets = self.targets[item]

    text = text + ' ' + auxiliary_sentence

    inputs = self.tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = self.max_len,
        pad_to_max_length = True
    )

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    return {
        "ids": torch.tensor(ids, dtype=torch.long),
        "mask": torch.tensor(mask, dtype=torch.long),
        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        "targets": torch.tensor(targets, dtype=torch.long),
        "opinions_id": torch.tensor(opinions_id, dtype=torch.long)
    }

In [11]:
test_dataset = SentiHood(
    opinions_id = test_data['id'].values,
    text = test_data['text'].values,
    auxiliary_sentence = test_data['auxiliary_sentence'],
    targets = test_data['sentiment'].values,
    tokenizer = tokenizer,
    max_len = max_length
)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = batch_size,
    shuffle=False
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [15]:
def infer_loop_function(data_loader, model, device):
  """
  This function performs the inference on testing sets and stores the predicted
  values.
  """

  model.eval()

  df_pred = pd.DataFrame({"id": [], "predicted": [], "actual": []})

  ii = 0
  for bi, d in tqdm(enumerate(data_loader), total=len(data_loader), ncols=80):
    opinions_id = d["opinions_id"]
    ids = d["ids"]
    mask = d["mask"]
    token_type_ids = d["token_type_ids"]
    targets = d["targets"]

    opinions_id = opinions_id.to(device, dtype=torch.long)
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    targets = targets.to(device, dtype=torch.long)

    outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
    _, predicted = torch.max(outputs, 1)
    
    predicted = predicted.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    opinions_id = opinions_id.detach().cpu().numpy()

    for k in range(len(predicted)):
      df_pred.loc[ii] = [str(opinions_id[k]), str(predicted[k]), str(targets[k])]
      ii += 1

    df_pred.to_csv('/content/PredictedValues.csv', index=False)

In [16]:
model = torch.load('/content/drive/My Drive/sentihood/1.bin')
infer_loop_function(data_loader=test_data_loader, model=model, device=device)

100%|███████████████████████████████████████████| 13/13 [04:05<00:00, 18.91s/it]


# Inference

In [17]:
def compute_sentiment_accuracy(df):
  accuracy = df[df['predicted'] == df['actual']].shape[0]/df.shape[0] * 100
  return round(accuracy, 2)

df = pd.read_csv('/content/PredictedValues.csv')
print(f'Sentiment Accuracy= {compute_sentiment_accuracy(df)}%')

Sentiment Accuracy= 92.25%


In [19]:
def compute_aspect_accuracy(df):
  """
  This function computes the strict aspect accuracy.
  0 => Represents that the aspect has not been detected.
  1 => Represents that the aspect has been detected.
  """
  
  df = df.replace([0, 1], 1).replace(2, 0)

  count = 0
  total = 0

  for i in range(0, df.shape[0], 12):
    true_values = df.iloc[i:i+12]['predicted']
    predicted_values = df.iloc[i:i+12]['actual']

    if (true_values == predicted_values).all():
      count += 1
    total += 1

  accuracy = float(count)/float(total)*100
  return round(accuracy, 2)

df = pd.read_csv('/content/PredictedValues.csv')
print(f'Aspect Accuracy= {compute_aspect_accuracy(df)}%')

Aspect Accuracy= 29.41%


In [20]:
def compute_aspect_f1_score(df):
  """
  This function computest the macro F1 score of predicted aspects.
  0 => Represents that the aspect has not been detected.
  1 => Represents that the aspect has been detected.
  """
  
  df = df.replace([0, 1], 1).replace(2, 0)

  total_f1_score = 0
  total = 0
  
  for i in range(0, df.shape[0], 12):
    true_values = df.iloc[i:i+12]['predicted']
    predicted_values = df.iloc[i:i+12]['actual']

    total_f1_score += f1_score(true_values, predicted_values, average="macro")
    total += 1

  score = float(total_f1_score)/float(total)*100
  return round(score, 2)

df = pd.read_csv('/content/PredictedValues.csv')
print(f"Aspect F1 score: {compute_aspect_f1_score(df)}")

Aspect F1 score: 62.33


# create json

In [21]:
df = pd.read_csv('/content/PredictedValues.csv')
"""
Computes the positive correct, positive total, negative correct, negative total, 
none correct, none total corresponding to all the aspects of LOCATION1.
"""

aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']
location1_aspects_result_analysis = {}

for i in range(12):
  location1_aspects_result_analysis[aspects[i]] = [[0 ,0], [0 ,0], [0 ,0]]

for i in tqdm(range(0, df['id'].unique().shape[0]*12-12, 12), ncols=80):
  for j in range(12):
    if df.loc[i+j]['actual'] == df.loc[i+j]['predicted']:
      location1_aspects_result_analysis[aspects[j]][int(df.loc[i+j]['actual'])][0] += 1
    
    location1_aspects_result_analysis[aspects[j]][int(df.loc[i+j]['actual'])][1] += 1

100%|███████████████████████████████████████████| 33/33 [00:00<00:00, 86.73it/s]


In [22]:
df_location_aspect = pd.DataFrame({"location": [], "aspect": [], "positive correct": [], "positive total": [], "negative correct": [], "negative total": [], "none correct": [], "none total": [],})

ii = 0
for key in location1_aspects_result_analysis.keys():
  df_location_aspect.loc[ii] = ['LOCATION1', f"{key}", 
                                location1_aspects_result_analysis[key][0][0], 
                                location1_aspects_result_analysis[key][0][1], 
                                location1_aspects_result_analysis[key][1][0], 
                                location1_aspects_result_analysis[key][1][1], 
                                location1_aspects_result_analysis[key][2][0], 
                                location1_aspects_result_analysis[key][2][1]]
  ii += 1
"""
Computes the positive correct, positive total, negative correct, negative total, 
none correct, none total corresponding to all the aspects of LOCATION2.
"""

aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']
location2_aspects_result_analysis = {}

for i in range(12):
  location2_aspects_result_analysis[aspects[i]] = [[0 ,0], [0 ,0], [0 ,0]]

for i in tqdm(range(df['id'].unique().shape[0]*12, df.shape[0]-12, 12), ncols=80):
  for j in range(12):
    if df.loc[i+j]['actual'] == df.loc[i+j]['predicted']:
      location2_aspects_result_analysis[aspects[j]][int(df.loc[i+j]['actual'])][0] += 1
    
    location2_aspects_result_analysis[aspects[j]][int(df.loc[i+j]['actual'])][1] += 1

0it [00:00, ?it/s]


In [23]:
for key in location2_aspects_result_analysis.keys():
  df_location_aspect.loc[ii] = ['LOCATION2', f"{key}", 
                                location2_aspects_result_analysis[key][0][0], 
                                location2_aspects_result_analysis[key][0][1], 
                                location2_aspects_result_analysis[key][1][0], 
                                location2_aspects_result_analysis[key][1][1], 
                                location2_aspects_result_analysis[key][2][0], 
                                location2_aspects_result_analysis[key][2][1]]
  ii += 1
df_location_aspect['positive percentage'] = round(df_location_aspect['positive correct']/df_location_aspect['positive total']*100, 2)
df_location_aspect['negative percentage'] = round(df_location_aspect['negative correct']/df_location_aspect['negative total']*100, 2)
df_location_aspect['none percentage'] = round(df_location_aspect['none correct']/df_location_aspect['none total']*100, 2)

df_location_aspect['total percentage'] = round((df_location_aspect['positive correct'] + df_location_aspect['negative correct'] + df_location_aspect['none correct'])/(df_location_aspect['positive total'] + df_location_aspect['negative total'] + df_location_aspect['none total'])*100, 2)
df_location_aspect

,location,aspect,positive correct,positive total,negative correct,negative total,none correct,none total,positive percentage,negative percentage,none percentage,total percentage
0,LOCATION1,dining,0.0,0.0,0.0,0.0,33.0,33.0,NaN,NaN,100.0,100.00
1,LOCATION1,general,0.0,11.0,0.0,4.0,18.0,18.0,0.0,0.0,100.0,54.55
2,LOCATION1,green-nature,0.0,0.0,0.0,0.0,33.0,33.0,NaN,NaN,100.0,100.00
3,LOCATION1,live,0.0,3.0,0.0,0.0,30.0,30.0,0.0,NaN,100.0,90.91
4,LOCATION1,multicultural,0.0,0.0,0.0,0.0,33.0,33.0,NaN,NaN,100.0,100.00
5,LOCATION1,nightlife,0.0,0.0,0.0,0.0,33.0,33.0,NaN,NaN,100.0,100.00
6,LOCATION1,price,0.0,2.0,0.0,2.0,29.0,29.0,0.0,0.0,100.0,87.88
7,LOCATION1,quiet,0.0,0.0,0.0,1.0,32.0,32.0,NaN,0.0,100.0,96.97
8,LOCATION1,safety,0.0,3.0,0.0,1.0,29.0,29.0,0.0,0.0,100.0,87.88
9,LOCATION1,shopping,0.0,0.0,0.0,0.0,33.0,33.0,NaN,NaN,100.0,100.00


In [ ]:
# "general" aspect of "Location1" is most accurately detected
#▪ What the points of failure for the model - which aspects does it perform poorly on.
# Model must be trained on full dataset firstly
# The model could be improved using a better data imbalance optimizer
